## Connection to the Azure SQL Database

Defined some variables to programmatically create the connection to the SQL Database.

In [ ]:
jdbcUsername = "feuplogin"
jdbcPassword = "Logproject33"
jdbcHostname = "intranet14.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "intranetfeupp14"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

## Read and Transform Data

### Calendar Dimension

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

import datetime
from pyspark.sql.functions import expr, sequence

# to have the months in portuguese - locale unsuported for having the months in portuguese
#import locale
#locale.setlocale(locale.LC_TIME, 'pt_PT.utf8')

i_date = spark.read.jdbc(url=jdbcUrl, table="(SELECT TOP(1) [DataInicio] FROM [stg].[EXT_TBL_IMPUTACAO_DETALHE] ORDER BY [DataInicio] ASC) AS query", properties=connectionProperties)

f_date = spark.read.jdbc(url=jdbcUrl, table="(SELECT TOP(1) [DataFim] FROM [stg].[EXT_TBL_ORCAMENTO] ORDER BY [DataFim] DESC) AS query", properties=connectionProperties)

initial_date = expr("to_timestamp('{}')".format(i_date.collect()[0][0]))
final_date = expr("to_timestamp('{}')".format(f_date.collect()[0][0]))

timestamps_array = sequence(initial_date, final_date, expr("interval 1 month")).alias("timestamps")

for row in spark.range(1).select(timestamps_array).collect():
    for timestamp in row.timestamps:
        if (timestamp.month >= 10):
            id = str(timestamp.year) + str(timestamp.month)
            id = int(id)
            year_month_num = str(timestamp.year) +"-" + str(timestamp.month)
        else:
            id = str(timestamp.year) + "0" + str(timestamp.month)
            id = int(id)
            year_month_num = str(timestamp.year) +"-0" + str(timestamp.month)
            
        #month_extensive = timestamp.strftime('%B') formats the date for just the month
        if timestamp.month == 1:
          month_extensive = 'Janeiro'
        elif timestamp.month == 2:
          month_extensive = 'Fevereiro'
        elif timestamp.month == 3:
          month_extensive = 'Março'
        elif timestamp.month == 4:
          month_extensive = 'Abril'
        elif timestamp.month == 5:
          month_extensive = 'Maio'
        elif timestamp.month == 6:
          month_extensive = 'Junho'
        elif timestamp.month == 7:
          month_extensive = 'Julho'
        elif timestamp.month == 8:
          month_extensive = 'Agosto'
        elif timestamp.month == 9:
          month_extensive = 'Setembro'
        elif timestamp.month == 10:
          month_extensive = 'Outubro'
        elif timestamp.month == 11:
          month_extensive = 'Novembro'
        elif timestamp.month == 12:
          month_extensive = 'Dezembro'

        month_abbreviated = month_extensive[:3]
        month_abbreviated_year = month_abbreviated + " " + str(timestamp.year)
        month_extensive_year = month_extensive + " " + str(timestamp.year)

        month_to_quarter = {1:1, 2:1, 3:1, 4:2, 5:2, 6:2, 7:3, 8:3, 9:3, 10:4, 11:4, 12:4}
        quarter = month_to_quarter.pop(timestamp.month)
        quarter_extensive = str(timestamp.year) + " Trimestre " + str(quarter)
        
        year_string = str(timestamp.year)
        month_string = str(timestamp.month)
        
        # Creates the dataframe with the data needed
        table = [(id, timestamp.month, month_abbreviated, month_extensive, timestamp.year, year_month_num, month_abbreviated_year, month_extensive_year, quarter, quarter_extensive)]
        df = spark.createDataFrame(table, ["ID_Calendario", "Mes", "Mes_Abrev", "Mes_Extenso", "Ano", "Ano_Mes_Num", "Mes_Abrev_E_Ano", "Mes_Extenso_E_Ano", "Trimestre_Num", "Trimestre"])
    
        print(str(id) + " | " + str(timestamp.month) + " | " + month_abbreviated + " | " + month_extensive + " | " + str(timestamp.year) + " | " + str(year_month_num) + " | " + month_abbreviated_year + " | " + month_extensive_year + " | " + str(quarter) + " | " + quarter_extensive)

        # Uploads the data to the dimension
        df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_CALENDARIO]", mode="append", properties=connectionProperties)

# for tasks without time designated

table = [(0, 0, '---', 'Não Definido', 0, '---', 'Não Definido', 'Não Definido', 0, 'Não Definido')]
df = spark.createDataFrame(table, ["ID_Calendario", "Mes", "Mes_Abrev", "Mes_Extenso", "Ano", "Ano_Mes_Num", "Mes_Abrev_E_Ano", "Mes_Extenso_E_Ano", "Trimestre_Num", "Trimestre"])
    
print(str(id) + " | " + str(month) + " | " + month_abbreviated + " | " + month_extensive + " | " + str(year) + " | " + str(year_month_num) + " | " + month_abbreviated_year + " | " + month_extensive_year + " | " + str(quarter) + " | " + quarter_extensive)

df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_CALENDARIO]", mode="append", properties=connectionProperties)

201002 | 2 | Fev | Fevereiro | 2010 | 2010-02 | Fev 2010 | Fevereiro 2010 | 1 | 2010 Trimestre 1
201003 | 3 | Mar | Março | 2010 | 2010-03 | Mar 2010 | Março 2010 | 1 | 2010 Trimestre 1
201004 | 4 | Abr | Abril | 2010 | 2010-04 | Abr 2010 | Abril 2010 | 2 | 2010 Trimestre 2
201005 | 5 | Mai | Maio | 2010 | 2010-05 | Mai 2010 | Maio 2010 | 2 | 2010 Trimestre 2
201006 | 6 | Jun | Junho | 2010 | 2010-06 | Jun 2010 | Junho 2010 | 2 | 2010 Trimestre 2
201007 | 7 | Jul | Julho | 2010 | 2010-07 | Jul 2010 | Julho 2010 | 3 | 2010 Trimestre 3
201008 | 8 | Ago | Agosto | 2010 | 2010-08 | Ago 2010 | Agosto 2010 | 3 | 2010 Trimestre 3
201009 | 9 | Set | Setembro | 2010 | 2010-09 | Set 2010 | Setembro 2010 | 3 | 2010 Trimestre 3
201010 | 10 | Out | Outubro | 2010 | 2010-10 | Out 2010 | Outubro 2010 | 4 | 2010 Trimestre 4
201011 | 11 | Nov | Novembro | 2010 | 2010-11 | Nov 2010 | Novembro 2010 | 4 | 2010 Trimestre 4
201012 | 12 | Dez | Dezembro | 2010 | 2010-12 | Dez 2010 | Dezembro 2010 | 4 | 2010 

### Productivity Classification Dimension

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()


intervals = ["[0%, 100%[", "100%", "]100%, 125%]", "]125%, 150%]", "]150%, 175%]", "]175%, +∞]"]
notes = ["Menos do Previsto", "Como Previsto", "Até 25% Horas Extra Consumidas", "Até 50% Horas Extra Consumidas", "Até 75% Horas Extra Consumidas", "Mais de 75% Horas Extra Consumidas"]
number_notes = [5, 4, 3, 2, 1, 0]

for i in range(6):
  df = spark.createDataFrame([(i, intervals[i], notes[i], number_notes[i])], ["ID_Classificacao_Produtividade", "Intervalo", "Nota", "Nota_Num"])
  df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_CLASSIFICACAO_PRODUTIVIDADE]", mode="append", properties=connectionProperties)
  print(str(i) + " | " + intervals[i] + " | " + notes[i] + " | " + str(number_notes[i]))

  i += 1

df = spark.createDataFrame([(6, 'Inválido', 'Inválida', -1)], ["ID_Classificacao_Produtividade", "Intervalo", "Nota", "Nota_Num"])
df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_CLASSIFICACAO_PRODUTIVIDADE]", mode="append", properties=connectionProperties)
print(str(6) + " | " + 'Inválido' + " | " + 'Inválida' + " | " + str(-1))

ModuleNotFoundError: No module named 'pyspark'

### State Dimension

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()
  
states_font = spark.read.jdbc(url=jdbcUrl, table="(select distinct [Estado] from [stg].[EXT_TBL_PROJETOS]) AS query", properties=connectionProperties).collect()

states = []

for row in states_font:
    if row['Estado'] not in states:
        states.append(row['Estado'])
        
df = spark.createDataFrame([(0, 'Não Definido')], ["ID_Estado", "Estado"])
df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_ESTADO]", mode="append", properties=connectionProperties)
print("0 | Não Definido")

for i in range(len(states)):   
  df = spark.createDataFrame([(i + 1, states[i])], ["ID_Estado", "Estado"])
  df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_ESTADO]", mode="append", properties=connectionProperties)
  print(str(i + 1) + " | " + states[i])
  
  i += 1

0 | Não Definido
1 | Iniciado
2 | Em garantia
3 | Finalizado
4 | Por Adjudicar
5 | Fechado
6 | Adjudicado, por iniciar


### Profile Dimension

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

profiles_font = spark.read.jdbc(url=jdbcUrl, table="(select distinct [Perfil] from [stg].[EXT_TBL_ORCAMENTO]) AS query", properties=connectionProperties).collect()

profiles = []

for row in profiles_font:
    if row['Perfil'] not in profiles:
        profiles.append(row['Perfil'])
      
# Sorts profiles by number
sorted_profiles = sorted(profiles, key=lambda x: int(x[0:2]))
  
df = spark.createDataFrame([(0, 'Não definido', 0, 'Não definida')], ["ID_Perfil", "Nome_Perfil", "Num_Perfil", "Posicao_Perfil"])
df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_PERFIL]", mode="append", properties=connectionProperties)

i = 1
for profile in sorted_profiles:
  if int(profile[0:2]) in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    print(str(i) + " | " + profile + " | " + str(int(profile[0:2])) + " | " + profile[4:])
    
    df = spark.createDataFrame([(i, profile, int(profile[0:2]), profile[4:])], ["ID_Perfil", "Nome_Perfil", "Num_Perfil", "Posicao_Perfil"])
    df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_PERFIL]", mode="append", properties=connectionProperties)
    
  else:
    print(str(i) + " | " + profile + " | " + str(int(profile[0:2])) + " | " + profile[5:])
    
    df = spark.createDataFrame([(i, profile, int(profile[0:2]), profile[5:])], ["ID_Perfil", "Nome_Perfil", "Num_Perfil", "Posicao_Perfil"])
    df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_PERFIL]", mode="append", properties=connectionProperties)
  
  i += 1

1 | 1 - Administrador | 1 | Administrador
2 | 1 - Sócio | 1 | Sócio
3 | 11 - Administrador | 11 | Administrador
4 | 12 - Administrador Operacional | 12 | Administrador Operacional
5 | 21 - Director | 21 | Director
6 | 22 - Guru | 22 | Guru
7 | 23 - Director Operacional | 23 | Director Operacional
8 | 31 - Gestor de Projecto | 31 | Gestor de Projecto
9 | 32 - Arquitecto Especialista | 32 | Arquitecto Especialista
10 | 41 - Consultor Sénior | 41 | Consultor Sénior
11 | 42 - Arquiteto | 42 | Arquiteto
12 | 43 - Consultor Operacional Sénior | 43 | Consultor Operacional Sénior
13 | 51 - Programador Sénior | 51 | Programador Sénior
14 | 52 - Consultor Operacional | 52 | Consultor Operacional
15 | 61 - Programador | 61 | Programador
16 | 62 - Operacional | 62 | Operacional
17 | 71 - Programador Júnior N2 | 71 | Programador Júnior N2
18 | 72 - Operacional Júnior | 72 | Operacional Júnior
19 | 81 - Programador Júnior N1 | 81 | Programador Júnior N1
20 | 91 - Estagiário Curricular | 91 | Estagiá

### Employee Dimension

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

employees_font = spark.read.jdbc(url=jdbcUrl, table="(select distinct [Username] FROM [stg].[EXT_TBL_IMPUTACAO_DETALHE]) AS query", properties=connectionProperties).collect()       # already returns the usernames sorted by the identifier number (ex: utilizador.231 - ordered by 231)

employees = []

for row in employees_font:
    if row['Username'] not in employees:
        employees.append(row['Username'])
  
df = spark.createDataFrame([(0, 'Não Definido')], ["ID_Funcionario", "Nome_Funcionario"])
df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_FUNCIONARIO]", mode="append", properties=connectionProperties)

i = 1
for employee in employees:
  print(str(i) + " | " + employee)

  df = spark.createDataFrame([(i, employee)], ["ID_Funcionario", "Nome_Funcionario"])
  df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_FUNCIONARIO]", mode="append", properties=connectionProperties)
  
  i += 1

1 | utilizador.211
2 | utilizador.213
3 | utilizador.214
4 | utilizador.215
5 | utilizador.217
6 | utilizador.219
7 | utilizador.221
8 | utilizador.222
9 | utilizador.223
10 | utilizador.224
11 | utilizador.225
12 | utilizador.227
13 | utilizador.228
14 | utilizador.229
15 | utilizador.231
16 | utilizador.233
17 | utilizador.235
18 | utilizador.237
19 | utilizador.238
20 | utilizador.239
21 | utilizador.240
22 | utilizador.241
23 | utilizador.242
24 | utilizador.243
25 | utilizador.244
26 | utilizador.245
27 | utilizador.246
28 | utilizador.247
29 | utilizador.248
30 | utilizador.249
31 | utilizador.250
32 | utilizador.251
33 | utilizador.252
34 | utilizador.254
35 | utilizador.255
36 | utilizador.256
37 | utilizador.257
38 | utilizador.258
39 | utilizador.259
40 | utilizador.260
41 | utilizador.261
42 | utilizador.262
43 | utilizador.263
44 | utilizador.264
45 | utilizador.265
46 | utilizador.266
47 | utilizador.267
48 | utilizador.268
49 | utilizador.269
50 | utilizador.270
51 | util

### Task Dimension

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

tasks = spark.read.jdbc(url=jdbcUrl, table="(select [ID], [CodigoProjeto], [Username], [Tarefa], [NHoras] from [stg].[EXT_TBL_HORASPREVISTAS]) AS query", properties=connectionProperties).collect()

'''
ids = tasks[i][0]
projects = tasks[i][1]
tasks = tasks[i][2]
expected_hours = tasks[i][3]'''

for task in tasks:
  #print(str(task[0]) + " | " + task[1] + " | " + task[2] + " | " + str(int(task[3])))
  
  id = task[0]
  
  project = task[1]
  
  employee_name = task[2]
  employee_number = task[2][11:]
  
  task_name = task[3]
  
  expected_hours = int(task[4])
  
  #project_task_hours = project + "_employee" + employee_number + "_" + str(expected_hours) + "hours"
  project_task_hours = project + " funcionário " + employee_number + " (" + str(expected_hours) + " horas)"
  
  
  print(str(id) + " | " + project + " | " + employee_name + " | " + task_name + " | " + str(expected_hours) + " | " + project_task_hours)
  
  df = spark.createDataFrame([(id, task_name, project_task_hours, expected_hours)], ["ID_Tarefa", "Nome_Tarefa", "Projeto_Funcionario_Horas", "Horas_Previstas_Tarefa"])
  df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_TAREFA]", mode="append", properties=connectionProperties)

df = spark.createDataFrame([(0, 'Não Definido', 'Não Definido', 0)], ["ID_Tarefa", "Nome_Tarefa", "Projeto_Funcionario_Horas", "Horas_Previstas_Tarefa"])
df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_TAREFA]", mode="append", properties=connectionProperties)

### Project Dimension

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

projects = spark.read.jdbc(url=jdbcUrl, table="(select P.[CodigoProjeto], [Departamento], [Area], sum(H.[NHoras]) as [HorasPrevistas] from [stg].[EXT_TBL_PROJETOS] as P join [stg].[EXT_TBL_HORASPREVISTAS] as H on P.[CodigoProjeto] = H.[CodigoProjeto] group by P.[CodigoProjeto], [Departamento],[Area]) AS query", properties=connectionProperties).collect()

''' for better vizualization
select P.[CodigoProjeto], [Departamento], [Area], sum(H.[NHoras]) as [HorasPrevistas]
from [stg].[EXT_TBL_PROJETOS] as P join [stg].[EXT_TBL_HORASPREVISTAS] as H
on P.[CodigoProjeto] = H.[CodigoProjeto]
group by P.[CodigoProjeto], [Departamento],[Area]
'''

# project not defined for tasks without project id

id = 0
project_code = "NAO.DEFINIDO"
project_name = "Projeto " + project_code
department = "Não Especificado"
area = "Não Especificada"
expected_hours = 0
#print(str(id) + " | " + project_code + " | " + project_name + " | " + department + " | " + area + " | " + str(expected_hours) + " horas previstas")

df = spark.createDataFrame([(id, project_code, project_name, department, area, expected_hours)], ["ID_Projeto", "Codigo_Projeto", "Nome_Projeto", "Departamento", "Area", "Horas_Previstas_Projeto"])
df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_PROJETO]", mode="append", properties=connectionProperties)


i = 1

for project in projects:
  id = i
  
  project_code = project[0] #CodigoProjeto
  
  project_name = "Projeto " + project_code

  if project[1] == None:
    department = "Não Especificado"
  else:
    department = project[1]

  if project[2] == None:
    area = "Não Especificada"
  else:
    area = project[2]
  
  expected_hours = project[3]  #NHoras from the join of PROJETOS with HORASPREVISTAS

  print(str(id) + " | " + project_code + " | " + project_name + " | " + department + " | " + area + " | " + str(expected_hours) + " horas previstas")

  df = spark.createDataFrame([(id, project_code, project_name, department, area, expected_hours)], ["ID_Projeto", "Codigo_Projeto", "Nome_Projeto", "Departamento", "Area", "Horas_Previstas_Projeto"])
  df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[DIM_PROJETO]", mode="append", properties=connectionProperties)

  i += 1


1 | 1001Id.2015.144 | Projeto 1001Id.2015.144 | IT  - Soluções Corporativas | SharePoint | 48.00 horas previstas
2 | ABR.2017.003 | Projeto ABR.2017.003 | Armis Brasil - Estrutura | Administrativo e Financeiro | 300.00 horas previstas
3 | ABR.2019.004 | Projeto ABR.2019.004 | Identity & Security | PT - BR | 3873.00 horas previstas
4 | ABR.2019.137 | Projeto ABR.2019.137 | Business Development for BR | Interno | 136.00 horas previstas
5 | ABR.2019.141 | Projeto ABR.2019.141 | ITS Solutions | PT-BR | 96.00 horas previstas
6 | ABR.2019.177 | Projeto ABR.2019.177 | Enterprise Solutions | Não Especificada | 291.00 horas previstas
7 | ABR.2019.238 | Projeto ABR.2019.238 | Sourcing - Interno | Identidade e Segurança | 300.00 horas previstas
8 | ABR.2019.278 | Projeto ABR.2019.278 | Structure | Administração | 24.00 horas previstas
9 | ABR.2019.355 | Projeto ABR.2019.355 | Identity & Security | Desenvolvimento de Negócio | 16.00 horas previstas
10 | ABR.2019.356 | Projeto ABR.2019.356 | Busine

### Task Fact

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

imputations = spark.read.jdbc(url=jdbcUrl, table="(select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as HorasRealizadas from [stg].[EXT_TBL_IMPUTACAO_DETALHE] group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID]) AS query", properties=connectionProperties).collect()

'''for better visualization
select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as horas_realizadas
from [stg].[EXT_TBL_IMPUTACAO_DETALHE]
group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID]
'''

i = 0
for imputation in imputations:
  # Collected Data:
  # [CodigoProjeto]   --imputation[0]
  # [Username]        --imputation[1]
  # [Ano]             --imputation[2]
  # [Mes]             --imputation[3]
  # [FK_TarefaID]     --imputation[4]
  # HorasRealizadas   --imputation[5]

  # Pretended Result:
  # [ID] [int] PRIMARY KEY,
	# [Horas_Realizadas] [int],
	# [ID_Projeto] [int],
	# [ID_Calendario] [int],
	# [ID_Tarefa] [int],
	# [ID_Funcionario] [int],
  # [ID_Perfil] [int],
	# [ID_Classificacao_Produtividade_Tarefa] [int]

  # ID
  id = i


  # Hours Performed
  hours_performed = imputation[5]
  

  # Project
  query = "(select [ID_Projeto] from [dwProdutividade].[DIM_PROJETO] where [Codigo_Projeto] = '" + imputation[0] + "') AS query"
  project = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
  if not project:
    project_id = 0
  else:
    project_id = project[0][0]


  # Calendar
  if not imputation[2] or not imputation[3]:
    calendar_id = 0
  else:
    query = "(select [ID_Calendario] from [dwProdutividade].[DIM_CALENDARIO] where [Mes] = " + str(imputation[3]) + " and [Ano] = " + str(imputation[2]) + ") AS query"
    calendar = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
    if not calendar:
      calendar_id = 0
    else:
      calendar_id = calendar[0][0]


  # Task
  if not imputation[4]:
    # go to the tasks table and look for the id of the task with the same CodigoProjeto and the same Username
    query = "(select [Tarefa] from [stg].[EXT_TBL_HORASPREVISTAS] where [CodigoProjeto] = '" + imputation[0] + "' and [Username] = '" + imputation[1] + "') AS query"
    task_info = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
    if not task_info:
      task_id = 0
      query = "(select [ID_Tarefa], [Horas_Previstas_Tarefa], [Nome_Tarefa] from [dwProdutividade].[DIM_TAREFA] where [ID_Tarefa] = " + str(task_id) + ") AS query"
      task = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()  # to be used in the productivity classification calculation
    else:
      query = "(select [ID_Tarefa], [Horas_Previstas_Tarefa], [Nome_Tarefa] from [dwProdutividade].[DIM_TAREFA] where [Nome_Tarefa] = '" + task_info[0][0] + "') AS query"
      task = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
      task_id = task[0][0]
  else:
    task_id = imputation[4] # check if it works well, if not use above process
    query = "(select [ID_Tarefa], [Horas_Previstas_Tarefa], [Nome_Tarefa] from [dwProdutividade].[DIM_TAREFA] where [ID_Tarefa] = " + str(task_id) + ") AS query"
    task = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()  # to be used in the productivity classification calculation


  # Employee
  query = "(select [ID_Funcionario] from [dwProdutividade].[DIM_FUNCIONARIO] where [Nome_Funcionario] = '" + imputation[1] + "') AS query"
  employee = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
  if not employee:
    employee_id = 0
  else:
    employee_id = employee[0][0]


  # Profile
  if not task:
    profile_id = 0
  else:
    #query = "(select distinct [Perfil] from [stg].[EXT_TBL_ORCAMENTO] where [CodigoProjeto] = '" + imputation[0] + "' and [Username] = '" + imputation[1] + "') AS query"
    query = "(select [Perfil] from [stg].[EXT_TBL_HORASPREVISTAS] as H join [stg].[EXT_TBL_ORCAMENTO] as O on H.[OrcamentoID] = O.[ID] where H.[CodigoProjeto] = '" + imputation[0] + "' and [Username] = '" + imputation[1] + "' and H.[Tarefa] = '" + task[0][2] + "') AS query"
    #task[0][2] -> Nome Tarefa == Tarefa em Horas Previstas
    profile_info = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
    if not profile_info:
      profile_id = 0
    else:
      query = "(select [ID_Perfil] from [dwProdutividade].[DIM_PERFIL] where [Nome_Perfil] = '" + profile_info[0][0] + "') AS query"
      profile = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
      if not profile:
        profile_id = 0
      else:
        profile_id = profile[0][0]

  ''' for better visualization
  select [Perfil]
  from [stg].[EXT_TBL_HORASPREVISTAS] as H join [stg].[EXT_TBL_ORCAMENTO] as O
  on H.[OrcamentoID] = O.[ID]
  where H.[CodigoProjeto] = '" + imputation[0] + "' and [Username] = '" + imputation[1] + "' and H.[Tarefa] = '" + task[0][2] + "'"
  # where H.[CodigoProjeto] = 'DGIE.2023.304' and [Username] = 'utilizador.355' and H.[Tarefa] = 'Tarefa - Utilizador #355'
  '''


  # Productivity Classification - in a global perspective, not in a month related one
  '''
  [0%, 100%[ - 5        - classification id 0
  100% - 4              - classification id 1
  ]100%, 125%] - 3      - classification id 2
  ]125%, 150%] - 2      - classification id 3
  ]150%, 175%] - 1      - classification id 4
  ]175%, +∞] - 0        - classification id 5
  '''

  if not task: #or expected == 0:
    classification_id = 6
  else:
    expected = task[0][1] 
    #realized = imputation[5] - monthly
    query = "(select [CodigoProjeto], [Username], [FK_TarefaID], sum([NHoras]) as HorasRealizadas from [stg].[EXT_TBL_IMPUTACAO_DETALHE] where [CodigoProjeto] = '" + imputation[0] + "' and [Username] = '" + imputation[1] + "' and [FK_TarefaID] = " + str(task_id) + " group by [CodigoProjeto], [Username], [FK_TarefaID]) AS query"
    realized = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()

    # verify if the condition of FK_TarefaID works

    ''' for better visualization
    select [CodigoProjeto], [Username], [FK_TarefaID], sum([NHoras]) as HorasRealizadas 
    from [stg].[EXT_TBL_IMPUTACAO_DETALHE] 
    group by [CodigoProjeto], [Username], [FK_TarefaID]
    order by [CodigoProjeto], [Username]
    '''

    ''' this query is for a month related calculus, if it was needed the hours realized in a certain month
    select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as HorasRealizadas 
    from [stg].[EXT_TBL_IMPUTACAO_DETALHE] 
    group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID]
    where [CodigoProjeto] = imputation[0] and [Username] = imputation[1] and [Ano] = str(imputation[2]) and [Mes] = str(imputation[3]) and [FK_TarefaID] = str(task_id)
    '''

    if expected == 0:
      classification_id = 6   # if it is not a value for hours expected
    else:
      if not realized:
        classification_id = 6
      else:
        quotient = realized[0][3] / expected #realized[0][3] is the realized hours sum column
        print(quotient)

        if quotient < 1:
          classification_id = 0
        elif quotient == 1:
          classification_id = 1
        elif quotient <= 1.25:
          classification_id = 2
        elif quotient <= 1.50:
          classification_id = 3
        elif quotient <= 1.75:
          classification_id = 4
        else: 
          classification_id = 5

  df = spark.createDataFrame([(id, hours_performed, project_id, calendar_id, task_id, employee_id, profile_id, classification_id)], ["ID", "Horas_Realizadas", "ID_Projeto", "ID_Calendario", "ID_Tarefa", "ID_Funcionario", "ID_Perfil", "ID_Classificacao_Produtividade_Tarefa"])
  df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[FACTO_TAREFA]", mode="append", properties=connectionProperties)

  print(str(id) + " | " + str(hours_performed) + " horas realizadas | " +  str(project_id) + " | " + str(calendar_id) + " | " + str(task_id) + " | " + str(employee_id) + " | " + str(profile_id) + " | " + str(classification_id))

  i += 1

1.00
0 | 40.00 horas realizadas | 1 | 201507 | 869 | 88 | 0 | 1
1.375
1 | 11.00 horas realizadas | 1 | 201507 | 870 | 336 | 0 | 3
2 | 5.00 horas realizadas | 0 | 201510 | 0 | 1 | 0 | 6
3 | 5.00 horas realizadas | 0 | 201510 | 0 | 1 | 0 | 6
4 | 13.00 horas realizadas | 0 | 201511 | 0 | 1 | 0 | 6
5 | 48.50 horas realizadas | 0 | 201512 | 0 | 1 | 0 | 6
6 | 23.00 horas realizadas | 0 | 201601 | 0 | 1 | 0 | 6
7 | 2.00 horas realizadas | 0 | 201602 | 0 | 1 | 0 | 6
8 | 8.00 horas realizadas | 0 | 201602 | 0 | 1 | 0 | 6
9 | 13.00 horas realizadas | 0 | 201603 | 0 | 1 | 0 | 6
10 | 17.00 horas realizadas | 0 | 201604 | 0 | 1 | 0 | 6
11 | 25.00 horas realizadas | 0 | 201605 | 0 | 1 | 0 | 6
12 | 14.00 horas realizadas | 0 | 201606 | 0 | 1 | 0 | 6
13 | 5.00 horas realizadas | 0 | 201607 | 0 | 1 | 0 | 6
14 | 10.00 horas realizadas | 0 | 201608 | 0 | 1 | 0 | 6
15 | 23.00 horas realizadas | 0 | 201609 | 0 | 1 | 0 | 6
16 | 2.00 horas realizadas | 0 | 201610 | 0 | 1 | 0 | 6
17 | 15.00 horas realizadas |

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()
'''
code = 'ABR.2016.016'
query = "(select [CodigoProjeto], [Username], [FK_TarefaID], sum([NHoras]) as HorasRealizadas from [stg].[EXT_TBL_IMPUTACAO_DETALHE] where [CodigoProjeto] = '" + code + "' and [Username] = 'utilizador.211' and [FK_TarefaID] = 0 group by [CodigoProjeto], [Username], [FK_TarefaID]) AS query"
realized = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()

print(realized[0][0])
print(realized[0][1])
print(realized[0][2])
print(realized[0][3])'''
'''
query = "(select [ID_Calendario] from [dwProdutividade].[DIM_CALENDARIO] where [Mes] = 5 and [Ano] = 2017) AS query"
calendar = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
print(calendar[0][0])'''
'''
query = "(select [ID_Funcionario] from [dwProdutividade].[DIM_FUNCIONARIO] where [Nome_Funcionario] = 'utilizador.213') AS query"
employee = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
print(str(employee[0][0]))'''

query = "(select [Perfil] from [stg].[EXT_TBL_HORASPREVISTAS] as H join [stg].[EXT_TBL_ORCAMENTO] as O on H.[OrcamentoID] = O.[ID] where H.[CodigoProjeto] = 'DGIE.2023.304' and [Username] = 'utilizador.355' and H.[Tarefa] = 'Tarefa - Utilizador #355') AS query"
profile_info = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
print(profile_info[0][0])
print(profile_info[0][0] == None)

21 - Director
False


### Project Fact

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Produtividade").getOrCreate()

# Total hours executed in each project in each month

projects_mensal = spark.read.jdbc(url=jdbcUrl, table="(select [CodigoProjeto], [Ano], [Mes], sum([HorasRealizadas]) as TotalHorasRealizadas from ( select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as HorasRealizadas from [stg].[EXT_TBL_IMPUTACAO_DETALHE] group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID]) as T group by [CodigoProjeto], [Ano], [Mes]) AS query", properties=connectionProperties).collect()

''' for better visualization
select [CodigoProjeto], [Ano], [Mes], sum([HorasRealizadas]) as TotalHorasRealizadas
from (
    select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as HorasRealizadas 
from [stg].[EXT_TBL_IMPUTACAO_DETALHE] 
group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID]) as T
group by [CodigoProjeto], [Ano], [Mes]
'''

i = 0

for project in projects_mensal:

  # Collected Data:
  # [CodigoProjeto]   --project[0]
  # [Ano]             --project[1]
  # [Mes]             --project[2]
  # HorasRealizadas   --project[3]

  # Pretended Result:
  # [ID] [int] PRIMARY KEY,
  # [Avanco] [int],
  # [Horas_Realizadas] [int],
  # [ID_Estado] [int],
  # [ID_Projeto] [int],
  # [ID_Calendario] [int],
  # [ID_Classificacao_Produtividade_Tarefa] [int]

  # ID
  id = i


  # Advance
  advance = 0
  month = project[2]
  year = project[1]
  min_year = 2010
  condition = True
  while year >= 2010 and condition:
    while month >= 1 and condition:
      query = "(select [Avanco] from [stg].[EXT_TBL_HISTORICO_AVANCOS] where [CodigoProjecto] = '" + project[0] + "' and MONTH(DataAvanco) = '" + str(month) + "' and YEAR(DataAvanco) = '" + str(year) + "') AS query"
      advance_info = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()

      if advance_info:
        advance = advance_info[0][0]
        condition = False

      month -= 1
    year -= 1


  # Hours Performed
  hours_performed = project[3]


  # State
  # it is a fixed value, it is available only on stg.EXT_TBL_PROJETOS table
  query = "(select [Estado] from [stg].[EXT_TBL_PROJETOS] where [CodigoProjeto] = '" + project[0] + "') AS query"
  state = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
  if not state:
    state_id = 0   # Undefined state for exceptional cases
  else:
    state_id = state[0][0]


  # Project
  query = "(select [ID_Projeto], [Horas_Previstas_Projeto] from [dwProdutividade].[DIM_PROJETO] where [Codigo_Projeto] = '" + project[0] + "') AS query"
  project_info = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()      # also useful for productivity classification calculus
  if not project_info:
    project_id = 0
  else:
    project_id = project_info[0][0]


  # Calendar
  if not project[1] or not project[2]:    # Year ; Month
    calendar_id = 0
  else:
    query = "(select [ID_Calendario] from [dwProdutividade].[DIM_CALENDARIO] where [Mes] = " + str(project[2]) + " and [Ano] = " + str(project[1]) + ") AS query"
    calendar = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
    if not calendar:
      calendar_id = 0
    else:
      calendar_id = calendar[0][0]


  # Productivity Classification
  query = "(select sum([HorasRealizadas]) as TotalHorasRealizadas from ( select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as HorasRealizadas from [stg].[EXT_TBL_IMPUTACAO_DETALHE] group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID]) as T where [CodigoProjeto] = '" + project[0] + "' group by [CodigoProjeto]) AS query"
  realized = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()

  ''' for better visualization
  select sum([HorasRealizadas]) as TotalHorasRealizadas
  from (
      select [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID], sum([NHoras]) as HorasRealizadas 
  from [stg].[EXT_TBL_IMPUTACAO_DETALHE] 
  group by [CodigoProjeto], [Username], [Ano], [Mes], [FK_TarefaID]) as T
  where [CodigoProjeto] = '" + project[0] + "'
  group by [CodigoProjeto]
  '''

  if not realized:
    classification_id = 6
  else:
    #query = "(select [Horas_Previstas_Projeto] from [dwProdutividade].[DIM_PROJETO] where [Codigo_Projeto] = '" + project[0] + "') AS query"
    #expected = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties).collect()
    #if not expected:
    if not project_info:       # declared before with the needed info
      classification_id = 6
    else:
      if project_info[0][1] == 0:
        classification_id = 6
      else:
        quotient = realized[0][0] / project_info[0][1]
        print(quotient)

        if quotient < 1:
          classification_id = 0
        elif quotient == 1:
          classification_id = 1
        elif quotient <= 1.25:
          classification_id = 2
        elif quotient <= 1.50:
          classification_id = 3
        elif quotient <= 1.75:
          classification_id = 4
        else: 
          classification_id = 5


  df = spark.createDataFrame([(id, advance, hours_performed, state_id, project_id, calendar_id, classification_id)], ["ID", "Avanco_Projeto", "Horas_Realizadas_Projeto", "ID_Estado", "ID_Projeto", "ID_Calendario", "ID_Classificacao_Produtividade_Projeto"])
  #df.write.jdbc(url=jdbcUrl, table="[dwProdutividade].[FACTO_PROJETO]", mode="append", properties=connectionProperties)

  print(str(id) + " | " + str(advance) + "% | " + str(hours_performed) + " horas realizadas | " + str(state_id) + " | " +  str(project_id) + " | " + str(calendar_id) + " | " + str(classification_id))

  i += 1

1.0625
0 | 0% | 51.00 horas realizadas | Fechado | 1 | 201507 | 2
1 | 0.00% | 10.00 horas realizadas | Fechado | 0 | 201510 | 6
2 | 0.00% | 13.00 horas realizadas | Fechado | 0 | 201511 | 6
3 | 0.00% | 48.50 horas realizadas | Fechado | 0 | 201512 | 6
4 | 0% | 23.00 horas realizadas | Fechado | 0 | 201601 | 6
5 | 0% | 10.00 horas realizadas | Fechado | 0 | 201602 | 6
6 | 0% | 13.00 horas realizadas | Fechado | 0 | 201603 | 6
7 | 0% | 17.00 horas realizadas | Fechado | 0 | 201604 | 6
8 | 0% | 25.00 horas realizadas | Fechado | 0 | 201605 | 6
9 | 0% | 14.00 horas realizadas | Fechado | 0 | 201606 | 6
10 | 0% | 5.00 horas realizadas | Fechado | 0 | 201607 | 6
11 | 0% | 10.00 horas realizadas | Fechado | 0 | 201608 | 6
12 | 0% | 23.00 horas realizadas | Fechado | 0 | 201609 | 6
13 | 0% | 17.00 horas realizadas | Fechado | 0 | 201610 | 6
14 | 0% | 19.00 horas realizadas | Fechado | 0 | 201611 | 6
15 | 0% | 18.00 horas realizadas | Fechado | 0 | 201612 | 6
0.6366666666666666666666666667
16 |